## Masks to Bounding boxes (semantic segmentation to object detection segmentetaion)

In [9]:
# Segmentation classes
list_of_classes = ['Background', 'Water', 'Building_No_Damage', 'Building_Minor_Damage', 
                   'Building_Major_Damage', 'Building_Total_Destruction', 'Vehicle', 'Road-Clear', 
                   'Road-Blocked', 'Tree', 'Pool']

In [10]:
import cv2
import numpy as np
import os
import traceback

In [19]:
def process_image(image_path, image_paths2):
    #print(f"Starting processing {image_path} and {image_paths2}")
    try:
        image1 = cv2.imread(image_path)
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        
        class_colors = {
            0: (0, 0, 0), # Background
            1: (255, 0, 0), # Water
            2: (0, 255, 0), # Building_No_Damage
            3: (0, 255, 255), # Building_Minor_Damage
            4: (0, 0, 255), # Building_Major_Damage
            5: (255, 0, 255), # Building_Total_Destruction
            6: (0, 165, 255), # Vehicle
            7: (42, 42, 165), # Road-Clear
            8: (105, 105, 105), # Road-Blocked
            9: (203, 192, 255), # Tree
            10: (255, 255, 0) # Pool
        }
        
        lut = np.zeros((256, 1, 3), dtype=np.uint8)
        for class_id, color in class_colors.items():
            lut[class_id] = color
        colored_image = cv2.LUT(image1, lut)

        image2 = cv2.imread(image_paths2)
        # Draw bounding boxes
        rotated_boxes = []
        for class_id, color in class_colors.items():
            if class_id != 0:  
                mask = cv2.inRange(colored_image, np.array(color), np.array(color))
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    rotated_rect = cv2.minAreaRect(contour)
                    rectangle_color = class_colors[class_id]
                    box = cv2.boxPoints(rotated_rect)
                    box = np.int0(box)
                    rotated_boxes.append((class_id, box))
                    for class_id, box in rotated_boxes:
                        rectangle_color = class_colors[class_id]
                        cv2.putText(image2, str(f"class:{class_id}"), (box[0][0], box[0][1]), cv2.FONT_HERSHEY_SIMPLEX, 3, rectangle_color, 3, cv2.LINE_AA)
                    cv2.drawContours(image2, [box], 0, rectangle_color, 12) 

        save_dir = r"M:\SOL\YOLO\Converted-val-org-img" #Save directory
        output_filename = os.path.basename(image_paths2)
        output_path = os.path.join(save_dir, os.path.splitext(output_filename)[0] + '_bbox.jpg')
        cv2.imwrite(output_path, image2)
        # print(f"Finished processing {image_path}")
    except Exception as e:
        print(f"Error processing {image_path} and {image_paths2}: {e}")
        traceback.print_exc()
        
def main():
    image_dir = r"M:\SOL\YOLO\val-label-img" #Enter labeled image directory
    image_dir2 = r"M:\SOL\YOLO\val-org-img" #Enter original image directory
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
    image_paths2 = [os.path.join(image_dir2, filename) for filename in os.listdir(image_dir2)]
    combined_paths = list(zip(image_paths, image_paths2))
    print(f"Found {len(combined_paths)} image pairs to process.")

    for image_path, image_paths2 in combined_paths:
        process_image(image_path, image_paths2)

if __name__ == '__main__':
    main()

Found 449 image pairs to process.
